In [ ]:
from langchain.tools import tool
from dotenv import load_dotenv
import os 
import requests
from langchain_huggingface import HuggingFaceEndpoint ,ChatHuggingFace
from langchain_core.messages import HumanMessage

In [12]:
load_dotenv()
key_currency_converter =  os.getenv("EXCHANGE_CURRENCY_API_KEY")
print(key_currency_converter)

4f4b810b556d43839b207f6c


Fucntion for Get the conversion factor from base to target currency


In [49]:

@tool
def get_currency_factor(base_currency : str , target_currency : str)->float:
    """ This function fetches the currency conversion factor between the given base currency and the target currency """
    url =  f'https://v6.exchangerate-api.com/v6/{key_currency_converter}/pair/{base_currency}/{target_currency}'
    resposne = requests.get(url)
    return resposne.json()
    print(resposne.json())

In [57]:
@tool
def convert(base_currency_value : int , conversion_rate : float)->float:
    """Given a currency conversion rate this fucntion calculate the total value of currency in target currency from base currency value"""
    return base_currency_value*conversion_rate

In [53]:
output = get_currency_factor.invoke({
    "base_currency" : 'USD' ,
    "target_currency" : 'INR'
})

conversion_rate = output.get("conversion_rate")

print(conversion_rate)


87.6968


In [58]:
convert.invoke({
    "base_currency_value" : 10 ,
    "conversion_rate" : conversion_rate 
})

876.968

ModelBinding with tools

In [63]:
llm  =  HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-R1-0528",
    task="text-generation"
)

model =  ChatHuggingFace(llm  =  llm)

binded_model =  model.bind_tools([get_currency_factor, convert])

calling binding model 

In [ ]:
messages  = []
user_query = 'Can you help on converting 100 USD to rupee '
intent_output =  binded_model.invoke(user_query)
messages.append(HumanMessage(user_query))
messages.append(intent_output)

print(intent_output)

content='<think>\nWe are given two functions: \n1. get_currency_factor: to get the conversion factor from base_currency to target_currency.\n2. convert: to convert an amount using a conversion rate.\n\nThe user wants to convert 100 USD to rupee (which is INR, Indian Rupee).\n\nSteps:\n1. Call get_currency_factor to get the conversion rate from USD to INR.\n   We\'ll set base_currency as "USD" and target_currency as "INR".\n\n2. Then use the convert function with base_currency_value = 100 and the conversion_rate obtained from the first step.\n\nBut note: the function convert requires two parameters: base_currency_value (which is 100) and conversion_rate (which we will get from the first function call).\n\nSo, we will break it down into two tool calls.\n\nFirst, we get the conversion rate for USD to INR.\n</think>\nI\'ll help you convert 100 USD to Indian Rupees (INR). I\'ll use the currency conversion tools to get the current exchange rate and perform the calculation.\n\nFirst, I\'ll fe

Output Generation